# Imports and Configurations

In [53]:
import pandas as pd
import numpy as np
import re
import nltk
from num2words import num2words

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.stem import PorterStemmer
from autocorrect import Speller
from sklearn.feature_extraction.text import CountVectorizer
from translate import translator

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline 
import seaborn as sns

nltk.download('rslp')

# dataset visibility
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

[nltk_data] Downloading package rslp to /Users/angelachow/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


# Functions

In [54]:
def remove_stop_words(text):
    for word in text:
        if word in stopwords.words('english'):
            text.remove(word)
    return text

stemmer = PorterStemmer()
spell = Speller('en')

def spell_and_stem_words(text):
    for i in range(len(text)):
        text[i] = stemmer.stem(spell(text[i]))
    return text

# Joining dataset

In [55]:
#final dataframe
df = pd.DataFrame()

In [56]:
# read all datasets

# orders dataset
orders = pd.read_csv('data/olist_orders_dataset.csv')

# order items dataset
items = pd.read_csv('data/olist_order_items_dataset.csv')

# sellers dataset
sellers = pd.read_csv('data/olist_sellers_dataset.csv')

# sellers dataset
customers = pd.read_csv('data/olist_customers_dataset.csv')

# reviews dataset
reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')

# products dataset
products = pd.read_csv('data/olist_products_dataset.csv')

# sellers dataset
payments = pd.read_csv('data/olist_order_payments_dataset.csv')

# categories dataset
categories = pd.read_csv('data/product_category_name_translation.csv')

In [57]:
df = orders

df = df.merge(customers, on='customer_id', how='left')
df = df.merge(payments, on='order_id', how='left')
df = df.merge(reviews, on='order_id', how='left')

df = df.merge(items, on='order_id', how='left')
df = df.merge(sellers, on='seller_id', how='left')
df = df.merge(products, on='product_id', how='left')
df = df.merge(categories, on='product_category_name', how='left')

#df.head()

In [58]:
df.to_csv('data/merged_dataset.csv')

# EDA

## Customers

In [ ]:
customers.info()
customers.isnull().sum()

In [ ]:
cust = customers["customer_unique_id"].nunique()
print(cust, "unique customers")

In [ ]:
cities = customers["customer_city"].nunique()
c1 = customers.groupby('customer_city')['customer_id'].nunique().sort_values(ascending=False)
print("There are",cities,"unique cities in the dataset. The Top 10 cities based on customers_id are:")
c2 = c1.head(10)
print(c2)
print("\nTop 10 cities covers", round(c2.sum()/customers.shape[0]*100,1),"percent of all the orders.")
plt.figure(figsize=(16,8))
c2.plot(kind="bar",rot=0)

In [ ]:
state = customers["customer_state"].nunique()
c1 = customers.groupby('customer_state')['customer_id'].nunique().sort_values(ascending=False)
print("There are",cities,"unique states in the dataset. The Top 5 states are:")
c2 = c1.head(5)
print(c2)
print("\nTop 10 states covers", round(c2.sum()/customers.shape[0]*100,1),"percent of all the orders.")
plt.figure(figsize=(16,8))
c2.plot(kind="bar",rot=0)

## Orders

In [ ]:
orders.head()

In [ ]:
orders.info()
customers.isnull().sum()

In [ ]:
orders.head()

In [ ]:
orders_mod = orders.copy()
orders_mod["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"], format='%Y-%m-%d %H:%M:%S')
orders_mod["order_delivered_carrier_date"] = pd.to_datetime(orders["order_delivered_carrier_date"], format='%Y-%m-%d %H:%M:%S')
orders_mod["order_delivered_customer_date"] = pd.to_datetime(orders["order_delivered_customer_date"], format='%Y-%m-%d %H:%M:%S')
orders_mod["order_estimated_delivery_date"] = pd.to_datetime(orders["order_estimated_delivery_date"], format='%Y-%m-%d %H:%M:%S')
orders_mod.dtypes

In [ ]:
orders_mod.head()

In [ ]:
##Viz on when purchases are made during period in dataset.
counts = orders_mod.set_index("order_purchase_timestamp").groupby(pd.Grouper(freq='D')).count()
fig = plt.figure(figsize=(16,8))
ax = fig.gca()
counts.plot(y = "order_id", use_index=True, ax=ax)

In [ ]:
##Identifies orderstatus distribution
orderstatus = orders["order_status"].nunique()
o1 = orders.groupby('order_status')['customer_id'].nunique().sort_values(ascending=False)
print("There are",orderstatus,"unique order_status in the dataset.")
o2 = o1.head(8)
print(o2)
o3 = o1.head(1)
print("\nDelivered status covers", round(o3.sum()/orders.shape[0]*100,1),"percent of all the orders.")
plt.figure(figsize=(16,8))
o2.plot(kind="bar",rot=0)

## Order Reviews

In [ ]:
reviews.info()
reviews.head()

In [ ]:
reviews.isnull().values.any()

In [ ]:
# Determine how many missing data instances
reviews.isnull().sum()

In [ ]:
# Address missing data - we can see that there is a review score but not necessarily a comment or a title. Should we make it "none?"

reviews.isnull().sum()

## Items

In [ ]:
items.info()
items.isnull().sum()

In [ ]:
sellerinfo = items["seller_id"].nunique()
o1 = items.groupby('seller_id')['order_id'].nunique().sort_values(ascending=False)
print("There are",items,"unique items in the dataset.")
o2 = o1.head(10)
print(o2)

print("\nTop 10 items covers", round(o2.sum()/orders.shape[0]*100,1),"percent of all the orders.")
plt.figure(figsize=(16,8))
o2.plot(kind="bar",rot=0)

## Sellers

In [ ]:
sellers.info()
sellers.isnull().sum()

In [ ]:
sellerstatus = sellers["seller_id"].nunique()
o1 = sellers.groupby('seller_city')['seller_id'].nunique().sort_values(ascending=False)
print("There are",sellerstatus,"unique sellers in the dataset.")
o2 = o1.head(8)
print(o2)

print("\nTop Seller by city covers", round(o2.sum()/orders.shape[0]*100,1),"percent of all the sellers.")
plt.figure(figsize=(16,8))
o2.plot(kind="bar",rot=0)

In [ ]:
sellerstatus = sellers["seller_id"].nunique()
o1 = sellers.groupby('seller_state')['seller_id'].nunique().sort_values(ascending=False)
print("There are",sellerstatus,"unique sellers in the dataset.")
o2 = o1.head(8)
print(o2)

print("\nTop Sellers by state represents", round(o2.sum()/orders.shape[0]*100,1),"percent of all the sellers.")
plt.figure(figsize=(16,8))
o2.plot(kind="bar",rot=0)

## Products

In [ ]:
products.info()
products.isnull().sum()
# need to fix missing data 610 is the same entry but missing description. 2 items also missing (1 competely missing all except ID
#other one is just missing weight, lenght, height, width) Dtypes changed from float64 to object as a result of replacing NA
products["product_category_name"].fillna("None", inplace = True)
products["product_name_lenght"].fillna("None", inplace = True)
products["product_description_lenght"].fillna("None", inplace = True)
products["product_photos_qty"].fillna("None", inplace = True)
products["product_weight_g"].fillna("0", inplace = True)
products["product_length_cm"].fillna("0", inplace = True)
products["product_height_cm"].fillna("0", inplace = True)
products["product_width_cm"].fillna("0", inplace = True)

In [ ]:
products.isnull().sum()

In [ ]:
most_product=df.groupby('product_category_name_english').aggregate({'order_id':'count'}).rename(columns={'order_id':'order_count'}).sort_values(by='order_count',ascending=False).reset_index()
most_product.head()

In [ ]:
### Visualising top 10 most bought product categories:
sns.barplot(x='product_category_name_english',y='order_count',data=most_product[:10],color="green")
plt.xlabel("Product Category")
plt.ylabel("Number of orders")
plt.title("Most bought product categories")
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
prodcat= df["product_id"].nunique()
o1 = df.groupby('product_category_name_english')['product_id'].nunique().sort_values(ascending=False)
print("There are",prodcat,"unique products in the dataset.")
o2 = o1.head(10)
print(o2)

print("\nTop 10 Products by category represent", round(o2.sum()/orders.shape[0]*100,1),"percent of all the products.")
plt.figure(figsize=(16,8))
o2.plot(kind="bar",rot=0)

## Payment

In [ ]:
payments.info()
payments.isnull().sum()

## Categories

In [ ]:
categories.info()
categories.isnull().sum()

### NLP

nltk.download('punkt')
nltk.download("stopwords")# NLP

In [ ]:
nltk.download('punkt')
nltk.download("stopwords")

In [76]:
# translated Comments to English
translated = pd.read_csv('reviews_MASTERLOW.csv')

In [74]:
english

0                                                Terrible
1                                 The piece did not serve
2       Missed 1 product and those receiving 1 broken ...
3       here you are describing as delivered only unti...
4       Canceled my purchase one day before delivery, ...
                              ...                        
1868    The chair came with factory defect, a loose to...
1869                     My product came spoiled and bad.
1870     As soon as receiving product I evaluate positive
1871    The product was not delivered, the company doe...
1872    My opinion is that I bought the product it had...
Name: English, Length: 1873, dtype: object

In [82]:
# we are going to work with only reviews dataset
#reviews= reviews.dropna()
#reviews.head(100)


english = translated['English']
# replace numbers of words
english = english.apply(lambda t: re.sub(r"(\d+)", lambda x: num2words(int(x.group(0))), str(t)))
# replace special characters
english = english.apply(lambda t: re.sub(r"[^a-zA-Z0-9]+", ' ', str(t)))
# make all letters in a lower case
english = english.apply(lambda t: str(t).lower())

# tokenize text and remove stop words
tokenized_reviews = english.apply(lambda t: word_tokenize(t))
tokenized_reviews = tokenized_reviews.apply(lambda t: remove_stop_words(t))

# spell check and stemming
tokenized_reviews = tokenized_reviews.apply(lambda t: spell_and_stem_words(t))

#tokenized_reviews.head(100)

reviews_text = tokenized_reviews.apply(lambda t: " ".join(t))


In [ ]:
''''
we are going to work with only reviews dataset
reviews = reviews.dropna()
#reviews.head(100)

reviews_only = reviews[reviews['review_score'] == 1]
reviews_only = reviews_only['review_comment_message']
# replace numbers of words
reviews_only = reviews_only.apply(lambda t: re.sub(r"(\d+)", lambda x: num2words(int(x.group(0))), t))
# replace special characters
reviews_only = reviews_only.apply(lambda t: re.sub('[^A-zÀ-ú\s]/gi', ' ', t))
# make all letters in a lower case
reviews_only = reviews_only.apply(lambda t: t.lower())

# tokenize text and remove stop words
tokenized_reviews = reviews_only.apply(lambda t: word_tokenize(t))
tokenized_reviews = tokenized_reviews.apply(lambda t: remove_stop_words(t))

# spell check and stemming
tokenized_reviews = tokenized_reviews.apply(lambda t: spell_and_stem_words(t))

#tokenized_reviews.head(100)

reviews_text = tokenized_reviews.apply(lambda t: " ".join(t))
'''''

In [92]:
# feature matrix
# creating the feature matrix
matrix = CountVectorizer(max_features=10000, ngram_range=(2, 2))
X = matrix.fit_transform(reviews_text).toarray()
matrix.vocabulary_
#
pd.set_option('display.max_rows', None)
phrases = pd.DataFrame()
phrases['phrases'] = matrix.get_feature_names()
phrases['frequency'] = X.sum(axis=0) 


In [93]:
#X.sum(axis=0) 
#phrases['frequency'] = X.sum(axis=0) 
#del phrases['English']
phrases['English'] = ''
#phrases['English'] = np.NaN
phrases.info()
#len(phrases)
phrases.to_csv("phrases_eng_2_2.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   phrases    10000 non-null  object
 1   frequency  10000 non-null  int64 
 2   English    10000 non-null  object
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [49]:
from googletrans import Translator
import time

translator = Translator()
translator.raise_Exception = True

#mini-batch loop. Loops 200 at a time. Hit failures with the translate API, switched to the slow iterrows loop below
#for i in range(0, len(reviews_low), 200):
    #print(i)
    #a = reviews_low.iloc[i:i+199]
    #reviews_low['English']=a['review_comment_message'].apply(translator.translate, src='pt', dest='en').apply(getattr, args=('text',))

# row by row to handle when API limits are hit...not the best performance...can probably add the try/except to the mini-batch above for better performance, assuming it calls the google API in bulk
for i, row in phrases.iterrows():
    # Skip rows that were successful from previous attempts
    # Blank text also seems to throw an error in the translator
    if (pd.isnull(row['English']) or row['English'] == '') and not pd.isnull(row['phrases']) and row['phrases'].strip() != '':
        try:
            #getattr is to retrieve the object value for 'text', returns None if 'text' is not in the object
            phrases.at[i, 'English'] = getattr(translator.translate(row['phrases'], src='pt', dest='en'), 'text')
        # Most likely the 429 status code error from too many attempts
        except Exception as e:
            print('Error on {}: {}'.format(i, row['phrases']))
            print(e)
            print('pausing for 1min...')
            time.sleep(60)
            print('trying again...')
            phrases.at[i, 'English'] = getattr(translator.translate(row['phrases'], src='pt', dest='en'), 'text')
            print('Success! Moving on.')


            

In [45]:
phrases.to_csv("phrases_low_1-1.csv")